Authors: Karen Stengel, Jakidxav
    
This Notebook will show how easy it is to load in a pre-trained model (or one save to a checkpoint), inspect its architecture, and continue training on data. This script also generates an ROC curve for a given batch of training. Here, we will be loading in a recurrent neural network (RNN) modeled after the Yann LeCun's [LeNet](https://www.researchgate.net/publication/2985446_Gradient-Based_Learning_Applied_to_Document_Recognition) architecture.

In [ ]:
from contextlib import redirect_stdout
import matplotlib
matplotlib.use('Agg')
import seaborn as sns 
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
%matplotlib inline

import numpy as np
import pickle
import os
import datetime
import time
import random
import sys

from keras.models import Sequential, Model, save_model, load_model
import keras.backend as K
from keras.layers import Dense, Activation, Conv2D, Input, AveragePooling2D, MaxPooling2D, Flatten, LeakyReLU, TimeDistributed, LSTM, Dropout, BatchNormalization
from keras.metrics import binary_accuracy
from keras.losses import  binary_crossentropy
from keras.regularizers import l2
from keras.optimizers import SGD, Adam
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model
import sklearn.metrics as skm
from tensorflow import metrics as tf

#import helper methods
from plotting import *
from helper_methods import *

In [0]:
#will allow for files to
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

#path for data output. each file should contain all used params after training and metrics
outputDir = "/Users/jakidxav/Desktop/Testing/data/Recur/longer/"
saved_model = '/Users/jakidxav/Desktop/Testing/data/Recur/cm/180720__40_7_rnn.h5'

#example for generating list of random numbers for grid search
# list = random.sample(range(min, max), numberToGenerate)

#hyperparameters and paramters
#SGD parameters
dropout = 0.5
momentum = 0.99

learningRate = [0.49,  0.123, 0.225, 0.357, 0.347, 0.123, 0.001, 0.011, 0.184, 0.49,  0.154, 0.032, 0.205, 0.052, 0.266, 0.388]
epochs = [213, 178, 250, 198, 233, 176, 158, 233, 156, 229, 219, 211, 182, 207, 247, 209]
learningRate = 0.123
epochs = 200
decay = 1e-4
batch = 128

boolNest = True

#parameters for conv/pooling layers
strideC = [5,5, 1]
strideP = [2,2]
kernel = [5, 5,1]
pool = [2,2]

#parameters for Adam optimizaiton
boolAdam = True #change to false if SGD is desired

beta_1= 0.9
beta_2= 0.999

epsilon=None
amsgrad=False

In [0]:
#main stuff
#this should read in each dataset and call the NN algorithms.
print("you are in main.")
outputFile = ""
bestFile = ""

#start setting up the name for the output file
date = datetime.datetime.now().strftime("%y%m%d_")
print(date)

#for all dataset directories & all data in each: need to walk through the various dierctories that contain each dataset
#can change this directory once run location on cheyenne is selected

outputDL = date + "_40_"
print(outputDL)

#filepalths for all data and labels for a given lead time
X_train_filename = '/Users/jakidxav/Desktop/Testing/40_lead/X_train/X_train.txt'
X_dev_filename = '/Users/jakidxav/Desktop/Testing/40_lead/X_dev/X_dev.txt'
X_val_filename = '/Users/jakidxav/Desktop/Testing/40_lead/X_val/X_val.txt'

Y_train_filename = '/Users/jakidxav/Desktop/Testing/Y/Y_train/station0/Y_train.txt'
Y_dev_filename = '/Users/jakidxav/Desktop/Testing/Y/Y_dev/station0/Y_dev.txt'
Y_val_filename = '/Users/jakidxav/Desktop/Testing/Y/Y_val/station0/Y_val.txt'

#open all data
with open(X_train_filename, 'rb') as f:
    train_data = pickle.load(f)

with open(X_dev_filename, 'rb') as g:
    dev_data = pickle.load(g)

with open(X_val_filename, 'rb') as h:
    test_data = pickle.load(h)

with open(Y_train_filename, 'rb') as i:
    train_label = pickle.load(i)

with open(Y_dev_filename, 'rb') as j:
    dev_label = pickle.load(j)

with open(Y_val_filename, 'rb') as k:
    test_label = pickle.load(k)

#reshape all data files.
train_data2 = train_data.reshape(-1,120,340, 1)
dev_data2 = dev_data.reshape(-1,120,340,1)
test_data2 = test_data.reshape(-1,120,340,1)

you are in main.
180720_
180720__40_


In [0]:
print("recurrent neural network")
outputFile = outputDir + outputDL + "_7_rnn"

#create and fill file with parameters and network info
file = open(outputFile + '.txt', "w+")
#writeFile(file,neuronLayer, iterations, boolLSTM, boolAdam, boolNest, drop, kernel, pool, strideC, strideP, momentum, decay, learnRate, b1, b2, epsilon, amsgrad, searchNum)

#load pre-saved model
recurModel = load_model(saved_model)

recurrent neural network


Now that we gave loaded in a model, we can inspect its architecture, peak at its weights, and look at hyperparameter values like optimizer and learning rate. We can then use our model to further train on our data and construct an ROC curve to check the model's performance. Lastly, we can save our updated model's architecture for further use.

In [0]:
print(recurModel.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 24, 68, 20)        520       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 12, 34, 20)        0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 12, 680)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 12, 60)            177840    
_________________________________________________________________
dropout_8 (Dropout)          (None, 12, 60)            0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 720)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 721       
Total para

In [0]:
print(recurModel.get_weights())

[array([[[[ -3.80695611e-01,  -7.74996102e-01,  -1.00353777e+00,
            6.51016682e-02,  -2.17978060e-01,  -4.51719522e-01,
            3.28835770e-02,  -6.22471943e-02,   1.74302116e-01,
            7.97466040e-01,  -1.12698719e-01,   1.72505394e-01,
            1.25612304e-01,   2.79105753e-01,   6.72185600e-01,
           -3.58802779e-03,   5.93608320e-01,   1.75909683e-01,
            4.25455481e-01,  -3.79231870e-01]],

        [[ -5.40488899e-01,  -9.80358303e-01,  -8.62866521e-01,
            1.24342009e-01,  -3.55388343e-01,  -5.81693649e-01,
           -1.04001239e-01,  -3.44386026e-02,   1.88961297e-01,
            6.10487938e-01,  -6.55999593e-03,   5.25822565e-02,
           -6.83181062e-02,   1.84246227e-01,   6.53494656e-01,
           -4.52262349e-02,   3.31816375e-01,   4.84503768e-02,
            6.84829354e-01,  -4.32295859e-01]],

        [[ -4.74056691e-01,  -9.51027095e-01,  -8.80973101e-01,
            3.03077519e-01,  -5.98020494e-01,  -5.99380434e-01,
     

In [0]:
print(recurModel.optimizer)

In [0]:
print(recurModel.optimizer.lr)

<tf.Variable 'Adam_1/lr:0' shape=() dtype=float32_ref>


In [0]:
#recurModel.compile(loss=binary_crossentropy,optimizer=opt_rnn,metrics=[binary_accuracy])
recur_hist = recurModel.fit(train_data2, train_label,batch_size=batch,epochs=epochs,verbose=2,validation_data=(dev_data2, dev_label))

Train on 1560 samples, validate on 240 samples
Epoch 1/200
 - 4s - loss: 0.2091 - binary_accuracy: 0.9186 - val_loss: 1.1197 - val_binary_accuracy: 0.7875
Epoch 2/200
 - 3s - loss: 0.2083 - binary_accuracy: 0.9244 - val_loss: 0.8514 - val_binary_accuracy: 0.7917
Epoch 3/200
 - 3s - loss: 0.2179 - binary_accuracy: 0.9109 - val_loss: 0.9946 - val_binary_accuracy: 0.7875
Epoch 4/200
 - 3s - loss: 0.2138 - binary_accuracy: 0.9135 - val_loss: 0.8487 - val_binary_accuracy: 0.7833
Epoch 5/200
 - 3s - loss: 0.2139 - binary_accuracy: 0.9096 - val_loss: 0.9043 - val_binary_accuracy: 0.7875
Epoch 6/200
 - 3s - loss: 0.1922 - binary_accuracy: 0.9231 - val_loss: 1.0719 - val_binary_accuracy: 0.7875
Epoch 7/200
 - 3s - loss: 0.2110 - binary_accuracy: 0.9186 - val_loss: 1.0706 - val_binary_accuracy: 0.7833
Epoch 8/200
 - 4s - loss: 0.2068 - binary_accuracy: 0.9212 - val_loss: 0.9988 - val_binary_accuracy: 0.7917
Epoch 9/200
 - 3s - loss: 0.2145 - binary_accuracy: 0.9147 - val_loss: 0.8488 - val_binar

Epoch 76/200
 - 3s - loss: 0.1385 - binary_accuracy: 0.9436 - val_loss: 1.1132 - val_binary_accuracy: 0.7875
Epoch 77/200
 - 3s - loss: 0.1509 - binary_accuracy: 0.9391 - val_loss: 1.0701 - val_binary_accuracy: 0.7958
Epoch 78/200
 - 3s - loss: 0.1886 - binary_accuracy: 0.9154 - val_loss: 1.1017 - val_binary_accuracy: 0.7917
Epoch 79/200
 - 3s - loss: 0.1687 - binary_accuracy: 0.9314 - val_loss: 1.0552 - val_binary_accuracy: 0.7708
Epoch 80/200
 - 3s - loss: 0.1613 - binary_accuracy: 0.9385 - val_loss: 1.1322 - val_binary_accuracy: 0.7750
Epoch 81/200
 - 3s - loss: 0.1711 - binary_accuracy: 0.9244 - val_loss: 1.0797 - val_binary_accuracy: 0.7875
Epoch 82/200
 - 3s - loss: 0.1728 - binary_accuracy: 0.9237 - val_loss: 0.9757 - val_binary_accuracy: 0.7875
Epoch 83/200
 - 3s - loss: 0.1830 - binary_accuracy: 0.9199 - val_loss: 1.0882 - val_binary_accuracy: 0.7917
Epoch 84/200
 - 3s - loss: 0.1820 - binary_accuracy: 0.9256 - val_loss: 0.9748 - val_binary_accuracy: 0.8042
Epoch 85/200
 - 3s 

Epoch 151/200
 - 3s - loss: 0.1370 - binary_accuracy: 0.9462 - val_loss: 0.8771 - val_binary_accuracy: 0.7792
Epoch 152/200
 - 3s - loss: 0.1256 - binary_accuracy: 0.9462 - val_loss: 1.0473 - val_binary_accuracy: 0.7875
Epoch 153/200
 - 3s - loss: 0.1521 - binary_accuracy: 0.9359 - val_loss: 0.9460 - val_binary_accuracy: 0.8000
Epoch 154/200
 - 3s - loss: 0.1278 - binary_accuracy: 0.9551 - val_loss: 1.0094 - val_binary_accuracy: 0.7833
Epoch 155/200
 - 3s - loss: 0.1319 - binary_accuracy: 0.9487 - val_loss: 1.0336 - val_binary_accuracy: 0.7917
Epoch 156/200
 - 3s - loss: 0.1453 - binary_accuracy: 0.9423 - val_loss: 0.9883 - val_binary_accuracy: 0.8000
Epoch 157/200
 - 3s - loss: 0.1449 - binary_accuracy: 0.9436 - val_loss: 0.9943 - val_binary_accuracy: 0.7875
Epoch 158/200
 - 3s - loss: 0.1365 - binary_accuracy: 0.9449 - val_loss: 0.9410 - val_binary_accuracy: 0.7875
Epoch 159/200
 - 3s - loss: 0.1444 - binary_accuracy: 0.9417 - val_loss: 1.0495 - val_binary_accuracy: 0.7875
Epoch 160/

`outputDir = "/Users/jakidxav/Desktop/Testing/data/Recur/cm/"`<br>
`outputDL = date + "_40_"`<br>
`outputFile = outputDir + outputDL + "_7_rnn"`<br>
`outputFile='/Users/jakidxav/Desktop/Testing/data/Recur/cm/data_40_7_rnn'`<br>

In [0]:
#calculate ROC info
train_pred = recurModel.predict(train_data2).ravel()
dev_pred = recurModel.predict(dev_data2).ravel()
fpr_train, tpr_train, thresholds_train = skm.roc_curve(train_label,train_pred)
fpr_dev, tpr_dev, thresholds_dev = skm.roc_curve(dev_label, dev_pred)

#save roc plot, save model output
rfile = open(outputFile + '_roc_vals.txt', "w+")
rocFile(rfile, fpr_train, tpr_train, fpr_dev, tpr_dev)
makePlots(recur_hist, outputFile, "LSTM Neural Net", fpr_train, tpr_train, fpr_dev, tpr_dev, train_pred, dev_pred)
recurModel.save(outputFile+ '.h5')